# **Vectorization**

**By**: Pedro Ruiz

---

En este notebook nos centraremos en la **vectorización** de datos, que consiste en tomar los resultados de clasificación y detección de objetos para generar representaciones vectoriales por vídeo. Estas representaciones serán utilizadas más adelante como entrada para clasificadores y sistemas de recomendación.


Este trabajo lo dividiremos en **cuatro secciones**:

1. **Importaciones**
2. **Configuración**
3. **Representaciones Vectoriales**
4. **Almacenamiento de Resultados**


## 1. Importaciones

In [1]:
import os
import configparser
import pandas as pd
from collections import defaultdict
import numpy as np

## 2. Configuración

In [2]:
notebook_dir = os.getcwd()  # En notebooks, usamos el directorio actual
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))

config = configparser.ConfigParser()
config_path = os.path.join(project_root, 'config', 'settings.conf')
config.read(config_path)

detection_results_semantic_path = os.path.join(project_root, config['data']['detection_results_semantic_path'])
output_vector_path = os.path.join(project_root, config['data']['output_vector_path'])

## 3. Representaciones Vectoriales  

### Identificación de Clases  

En esta etapa, nos centraremos en **extraer todas las clases detectadas** en el proceso de clasificación. Para ello, **recorreremos los archivos CSV** y los **almacenaremos** para poder **asignarles un índice**.

In [3]:
categories = [dr for dr in os.listdir(detection_results_semantic_path) if os.path.isdir(os.path.join(detection_results_semantic_path, dr))]         # Lista de categorías (directorios) en la ruta de resultados semánticos
all_classes = set()                                                                                                                                 # Conjunto para almacenar todas las clases únicas

for category in categories:                                                                                                                         # Iterar sobre cada categoría
    category_path = os.path.join(detection_results_semantic_path, category)                                                                         # Ruta completa de la categoría
    for csv_file in os.listdir(category_path):                                                                                                      # Iterar sobre cada archivo CSV en la categoría
        if csv_file.endswith('.csv'):                                                                                                               # Verificar si el archivo es un CSV
            df = pd.read_csv(os.path.join(category_path, csv_file))                                                                                 # Leer el archivo CSV en un DataFrame
            if 'object' in df.columns:                                                                                                              # Verificar si la columna 'object' existe en el DataFrame
                df['object'] = df['object'].astype(str).fillna('unknown')                                                                           # Convertir la columna 'object' a string y llenar valores faltantes con 'unknown'
                objs = df['object'].unique()                                                                                                        # Obtener los valores únicos de la columna 'object'
                all_classes.update(objs)                                                                                                            # Actualizar el conjunto de todas las clases con los objetos encontrados
            else:
                print(f"{csv_file} no tiene columna 'object'.")

# LKista ordenada
all_classes = sorted(list(all_classes))                                                                                                             # Convertir el conjunto de clases a una lista ordenada
class_to_idx = {c: i for i, c in enumerate(all_classes)}                                                                                            # Crear un diccionario que mapea cada clase a un índice

print(f"Clases encontradas: {all_classes}")


Clases encontradas: ['airplane', 'apple', 'backpack', 'banana', 'baseball bat', 'baseball glove', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'cell phone', 'chair', 'class_87', 'class_88', 'class_89', 'class_90', 'clock', 'couch', 'cow', 'cup', 'dining table', 'dog', 'donut', 'elephant', 'fire hydrant', 'frisbee', 'giraffe', 'hair drier', 'horse', 'hot dog', 'keyboard', 'kite', 'knife', 'microwave', 'motorcycle', 'mouse', 'nan', 'orange', 'oven', 'parking meter', 'person', 'pizza', 'potted plant', 'refrigerator', 'sandwich', 'sheep', 'sink', 'skateboard', 'skis', 'snowboard', 'spoon', 'sports ball', 'stop sign', 'suitcase', 'surfboard', 'teddy bear', 'tennis racket', 'toaster', 'toothbrush', 'traffic light', 'train', 'truck', 'umbrella', 'vase', 'wine glass', 'zebra']


### Cálculo de Vectores  

A continuación, generamos un **vector de características** para cada uno de los vídeos. Cada posición corresponde a una **clase identificada**, y su valor dependerá de:  

- La **frecuencia** con la que apareció cada objeto.  
- La **confianza** de la detección.  
- El **área promedio** del objeto. 

In [4]:
video_features = []
video_labels = []

for category in categories:                                                             # Iteramos  sobre las categoría
    category_path = os.path.join(detection_results_semantic_path, category)             # y creamos la ruta.
    for csv_file in os.listdir(category_path):                                          # Iteramoes sobre cada archivo CSV,
        if csv_file.endswith('.csv'):                                                   # verificando que sean CSVs
            video_name = csv_file.replace('.csv', '')                                   # extraemos el nombre del video
            df = pd.read_csv(os.path.join(category_path, csv_file))                     # y los leemos.
            
            # Comprobación de columnas 'object' y gestión de valores faltantes
            if 'object' in df.columns:
                df['object'] = df['object'].astype(str).fillna('unknown')
            else:
                print(f"Advertencia: {csv_file} no contiene la columna 'object'. Aplicando 'unknown' sobre todos los objetos.") 
                df['object'] = 'unknown'
            
            # Crear vector de features
            vec = np.zeros(len(all_classes), dtype=float)                               # Inicializamos un vector de ceros para todas las clases
            
            # Agrupar por objeto
            class_groups = df.groupby('object')                                         # Agrupamos el conjunto por la columna 'object'
            for obj, group in class_groups:                                             # e iteramos sobre cada grupo de objetos
                idx = class_to_idx.get(obj, None)                                       # extraemos el índice del objeto en el diccionario class_to_idx
                if idx is not None:                                                     # Verificamos que el objeto exista en el diccionario
                    freq = len(group)                                                   # y calculamos la frecuencia de aparición, la confianza promedio y el área promedio.
                    mean_conf = group['score'].mean() if 'score' in group.columns else 0.0
                    mean_area = group['area'].mean() if 'area' in group.columns else 0.0
                    importance = freq * mean_conf * (mean_area / (224*224))             # Ademas, incorporamos el calculo de la importancia normalizando el área
                    vec[idx] = importance                                               # y la asignamos al vector en la posición del índice.
                else:
                    print(f"'{obj}' no encontrado en class_to_idx.")
            
            # Guardar el vector
            video_features.append([category, video_name] + vec.tolist())                # Añadir el vector de características a la lista video_features
            video_labels.append(category)                                               # Añadir la categoría a la lista video_labels

print(f"Se calcularon vectores para {len(video_features)} vídeos.")

Se calcularon vectores para 300 vídeos.


## 4. Almacenamiento

In [6]:
columns = ['category', 'video_name'] + all_classes
vectors_df = pd.DataFrame(video_features, columns=columns)
display(vectors_df.head())
vectors_df.to_csv(output_vector_path, index=False)

,category,video_name,airplane,apple,backpack,banana,baseball bat,baseball glove,bear,bed,...,tennis racket,toaster,toothbrush,traffic light,train,truck,umbrella,vase,wine glass,zebra
0,cooking,cooking_000001,0.0,5.270265,0.0,9.568111,0.039174,0.009999,0.0,1.717104,...,0.030303,0.00000,12.608868,0.247205,0.000000,0.0,0.000000,5.098269,0.033967,0.0
1,cooking,cooking_000002,0.0,3.763367,0.0,9.612788,0.061967,0.000000,0.0,3.518374,...,0.586366,1.51012,15.987610,0.000000,0.000000,0.0,0.039186,7.786044,0.191073,0.0
2,cooking,cooking_000003,0.0,6.713740,0.0,0.694836,0.000000,0.000000,0.0,0.864996,...,0.000000,0.00000,3.816146,0.000000,0.000000,0.0,0.000000,0.291463,0.000000,0.0
3,cooking,cooking_000004,0.0,1.179637,0.0,2.123778,0.000000,0.000000,0.0,9.219209,...,0.000000,0.00000,9.287340,0.000000,0.066707,0.0,0.000000,2.291315,0.000000,0.0
4,cooking,cooking_000005,0.0,6.619722,0.0,0.613203,0.000000,0.000000,0.0,1.421126,...,0.000000,0.00000,1.548106,0.000000,0.000000,0.0,0.000000,1.785013,0.002377,0.0


In [17]:
columns = ['category','video_name'] + all_classes
vectors_df = pd.DataFrame(video_features, columns=columns)
output_vector_path = "../data/video_features.csv"
vectors_df.to_csv(output_vector_path, index=False)
print(f"Vectores guardados en {output_vector_path}")

Vectores guardados en ../data/video_features.csv
